# **Project Details:**

Gestore delle spese domestiche

Context:
In a society where financial management is crucial, an app for tracking personal and family expenses becomes essential. This project aims to provide a simple app to track daily expenses.

Objective:
To develop an app in Python to record expenses, generate monthly reports and identify the most significant expenses, helping users to manage the family budget consciously.

Key Features:

Adding a Transaction: Users can record expenses with date, description and amount, saved in a CSV file.
Monthly Report: Generate reports that group expenses by month and year.
Top 10 Transactions: Display of the top 10 expenses to monitor relevant expenses.
Added Value:

Instant Monitoring: Real-time tracking of expenses.
Conscious Budget Management: Aggregated view of expenses to better understand money flows.
Identification of Major Expenses: Top 10' function to identify major expenses.
Technical Implementation:
The app uses Python and CSV files for data management, with an interactive menu to guide the user.

Conclusion:
This app offers users an easy way to monitor finances, promoting transparency and efficiency in managing personal or family budgets.

# **Importing Necessary Modules, Setting Constants and Functions**

## *Necessary Modules*

In [ ]:
import csv
from datetime import datetime
import re
import os
from typing import Dict, Union, Optional

## *Constants*

In [ ]:
YES_INPUT = ["yes", "y", "si", "s", "ok"]
NO_INPUT = ["no", "n", "nope", "not"]
CORRECT_HEADERS = ['Date', 'Description', 'Amount']
BYE_MESSAGE = "Goodbye, hope to see you soon!"
WELCOME_MESSAGE = """Hello and welcome!
This program allows you to create a CSV file containing the date, description and amount of the transactions made.
It can also generate monthly reports of transactions made and show the 10 transactions with the highest amount.
Have fun with it!"""

## *Functions*

In [ ]:
#'Check data' functions

def check_date(date: str) -> bool:

    """
    Checks if the given date string is in the correct format and represents a valid date.

    Args:
        date (str): A string representing a date in the format "%d/%m/%Y".

    Returns:
        bool: True if the date is in the correct format and exists, False otherwise.
    """

    try:
        datetime.strptime(date, "%d/%m/%Y")
    except ValueError:
        print("The date format is incorrect. Check if the date exists.")
        return False
    return True


def check_description(description: str) -> bool:

    """
    Checks if the description (string) is not empty by eliminating whitespace from string.

    Args:
        description (str): A string representing a text description.

    Returns:
        bool: True if the description is not empty, False otherwise.
    """

    if not description.strip():
        print("Description is empty.")
        return False
    return True


def check_amount(amount: str) -> bool:

    """
    Checks if the given amount string is in the correct format, converting it to a floating-point number.

    Args:
        amount (str): A string representing the amount, which can include numbers, '.' or ','.

    Returns:
        bool: True if the amount can be converted to a float, False otherwise.
    """

    amount = amount.replace(",", ".")
    try:
        float(amount)
        return True
    except ValueError:
        print("The amount format is not correct. Amount should made up of numbers, '.' or ','")
        return False


def convert_numbers(transaction: Dict[str, str]) -> Dict[str, Union[int, float]]:

    """
    Converts the 'Amount' field in the transaction dictionary from a string to a number (either an integer or a float).

    Args:
        transaction (Dict[str, str]): A dictionary representing a transaction, where
                                      'Amount' is one of the keys and its value is a string
                                      representing a numerical value.

    Returns:
        Dict[str, Union[int, float]]: The updated transaction dictionary with the
                                      'Amount' field converted to an integer if it's a
                                      whole number, or to a float if it contains decimals.

    """

    amount = transaction["Amount"]
    transaction["Amount"] = int(amount) if amount.isdecimal() else float(amount)
    return transaction


#'Check CSV file' functions

def check_existence_file(complete_filepath: str) -> bool:
    """
    Checks if the file exists and if it has the CSV extension.

    Args:
        complete_filepath (str): The path to the file to be checked.

    Returns:
        bool: True if the file exists and has a CSV extension, False otherwise.
    """

    if not os.path.isfile(complete_filepath):
        print("The file does not exist.")
        return False
    if not complete_filepath.lower().endswith('.csv'):
        print("File is not in CSV format. Check the file extension.")
        return False
    return True


def check_structure_file(complete_filepath: str) -> bool:
    """
    Checks if the CSV file has a valid structure.

    Args:
        complete_filepath (str): The path to the file to be checked.

    Returns:
        bool: True if the file has a valid structure, False otherwise.
    """

    try:
        with open(complete_filepath, 'r', newline='', encoding='utf-8') as file:
            csv_reader = csv.reader(file)
            headers = next(csv_reader, None)
            if headers is None:
                print("The CSV file is empty.")
                return False
            if headers != CORRECT_HEADERS:
                print(f"Invalid CSV file structure. Expected headers: {CORRECT_HEADERS}")
                return False

            rows = list(csv_reader)
            if not rows:
                print("The CSV file contains only the headers. You can add new rows choosing option 1.\nPay attention: 'Monthly report' and 'Top ten transactions' will be empty.")
                return True

            for i, row in enumerate(rows, start=1):
                if len(row) != 3:
                    print(f"Invalid CSV file structure: row {i} does not have 3 columns.")
                    return False
                date, description, amount = row
                if not (check_date(date) and check_description(description) and check_amount(amount)):
                    print(f"Invalid data in row {i}")
                    return False
        return True
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return False


def verify_csv_file(complete_filepath: str) -> bool:
    """
    Verifies both the existence and the structure of the CSV file.

    Args:
        complete_filepath (str): The path to the file to be checked.

    Return:
        bool: True if both 'check_esistence_file' and 'check_structure_file' do not raise exceptions, False otherwise.
    """

    return check_existence_file(complete_filepath) and check_structure_file(complete_filepath)


#'Check transaction' functions

def extract_transaction_data(user_transaction: str) -> tuple[bool, Optional[str], Optional[str], Optional[str]]:
    """
    Verifies and extracts the date, description, and amount from a user transaction.

    Args:
        user_transaction (str): The user transaction input string in the format 'DD/MM/YYYY description amount'.

    Returns:
        tuple[bool, Optional[str], Optional[str], Optional[str]]:
            A tuple where the first element is a boolean indicating success, and the following elements are the extracted date, description, and amount respectively.
    """

    pattern = r'(\d{2}/\d{2}/\d{4})\s+(.+?)\s+(\d+(?:[.,]\d{1,})?)$'
    matches = re.findall(pattern, user_transaction)

    if not matches:
        print("The transaction format is not correct. Should be 'DD/MM/YYYY' 'description not empty' 'amount should be composed by numbers, '.' or ',''")
        return False, None, None, None

    date, description, amount = matches[0]
    if not (check_date(date) and check_description(description) and check_amount(amount)):
        return False, None, None, None

    return True, date, description, amount.replace(",", ".")


#'Manage Transaction to or from CSV functions'

def transaction_to_csv(complete_filepath: str, user_transaction: str) -> None:
    """
    Adds a valid transaction to a CSV file. If the file does not exist, it creates a new file with the header.

    Args:
        complete_filepath (str): The path to the CSV file.
        user_transaction (str): The user transaction input string in the format 'DD/MM/YYYY description amount'.

    Returns:
        None
    """

    is_right, date, description, amount = extract_transaction_data(user_transaction)
    if not is_right:
        print("The transaction will not be added to the file.")
        return

    with open(complete_filepath, 'a', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        if file.tell() == 0:
            csv_writer.writerow(CORRECT_HEADERS)
        csv_writer.writerow((date, description, amount))
    print("Transaction added successfully.")


def transaction_from_csv(complete_filepath: str) -> Optional[list[Dict[str, str]]]:
    """
    Checks the CSV file. If it is correct, it reads the file as a dictionary and extracts the transactions as a list of dictionaries.

    Args:
        complete_filepath (str): The path to the file to be checked.

    Returns:
        bool: True if both 'check_existence_file' and 'check_structure_file' do not raise exceptions, False otherwise.
    """

    if verify_csv_file(complete_filepath):
      with open(complete_filepath, 'r', newline='', encoding='utf-8') as file:
          transactions = list(csv.DictReader(file))
          return transactions
    return None


#'Generate report' functions

def transaction_report(complete_filepath: str) -> None:
    """
    Generates a monthly report of transactions. It reads the CSV file, sorts the transactions by date and prints the transactions grouped by year-month.

    Args:
        complete_filepath (str): The path to the CSV file.

    Returns:
        None
    """

    try:
      transactions = transaction_from_csv(complete_filepath)
      transactions.sort(key=lambda x:datetime.strptime(x["Date"], "%d/%m/%Y"))

      monthly_report = {}
      for transaction in transactions:
          year_month = datetime.strptime(transaction['Date'], "%d/%m/%Y").strftime("%Y-%m")
          if year_month not in monthly_report:
              monthly_report[year_month] = 0.0
          monthly_report[year_month] += float(transaction['Amount'])

      print("\nMonthly report: ")
      for year_month, total_amount in monthly_report.items():
          print(f"{year_month}: {total_amount:.2f}")

    except Exception as e:
          raise Exception(f"Unexpected error while generating report: {e}")


def top_ten_transaction(complete_filepath: str) -> None:
    """
    Returns the 10 transactions with the highest amount. Reads the CSV file, sorts the transactions by amount in descending order and prints the top ten.

    Args:
        complete_filepath (str): The path to the CSV file.

    Returns:
        None
    """

    try:
      transactions = transaction_from_csv(complete_filepath)
      transactions = [convert_numbers(transaction) for transaction in transactions]
      transactions.sort(key = lambda x: x["Amount"] , reverse=True)
      top_ten = transactions[:10]

      print("\nThe top ten transactions are:")
      for trans in top_ten:
          print(f"{trans['Date']} {trans['Description']} {trans['Amount']}")

    except Exception as e:
        raise Exception(f"Unexpected error while generating report: {e}")


#'Manage user interactions' functions

def get_csv_filepath() -> Optional[str]:
    """
    Obtains a CSV file path from the user. The user has three options:
    confirm if the CSV file already exists, provide a path to create a new CSV file, or exit the program.

    Returns:
        Optional[str]: The complete file path to the CSV file or None if the user chooses to exit.

    """

    while True:
        user_input = input("\nCSV file already exist? ['y'/'n' or 'a' to exit the program]: ").strip().lower()   #.strip() and .lower() are used to avoid typing errors
        if user_input in YES_INPUT:
            complete_filepath = input("\nPlease, enter the file path or name (valid option only if the CSV file is in the same folder as the notebook): ").strip()
            if verify_csv_file(complete_filepath):
                return complete_filepath
        elif user_input in NO_INPUT:
            filepath = os.path.abspath(os.getcwd())
            complete_filepath = f"{filepath}/Gestione_spese_domestiche.csv"
            try:
                with open(complete_filepath, 'w') as file:
                    file.write("Date,Description,Amount\n")
                return str(complete_filepath)
            except Exception as e:
                print(f"Unexpected error: {e}")
                print("Unable to create the file. Try again or exit the program.")
        elif user_input == "a":
            return None
        else:
            print("Invalid option. Please try again.")

#'Main Code' function

def main_menu(complete_filepath: str) -> None:
    """
    Presents a menu of options to the user and performs specific actions based on the user's choice.

    Args:
        complete_filepath (str): The path to the CSV file.

    Returns:
        None
    """

    while True:
        user_option = input("\nInsert a number for the action to be executed:\n"
                          "[1] Enter a transaction\n"
                          "[2] Monthly report\n"
                          "[3] Top ten transactions\n"
                          "[4] Exit the programme\n").strip()
        if user_option == "1":
            user_transaction = input("Please, enter date [DD/MM/YYYY], description and amount"
                                    "[e.g. 18/05/2024 Dinner at restaurant 45.88]:\n ").strip()
            transaction_to_csv(complete_filepath, user_transaction)
        elif user_option == "2":
            transaction_report(complete_filepath)
        elif user_option == "3":
            top_ten_transaction(complete_filepath)
        elif user_option == "4":
            print(BYE_MESSAGE)
            break
        else:
            print("The number entered does not match any option. Please try again.")

# **Main Code**

In [ ]:
#Print welcome message
print(WELCOME_MESSAGE)


complete_filepath = get_csv_filepath()
if complete_filepath is None:
    print(BYE_MESSAGE)
else:
  main_menu(complete_filepath)

Hello and welcome!
This program allows you to create a CSV file containing the date, description and amount of the transactions made.
It can also generate monthly reports of transactions made and show the 10 transactions with the highest amount.
Have fun with it!

CSV file already exist? ['y'/'n' or 'a' to exit the program]: y

Please, enter the file path or name (valid option only if the CSV file is in the same folder as the notebook): /content/Gestione_spese_domestiche.csv

Insert a number for the action to be executed:
[1] Enter a transaction
[2] Monthly report
[3] Top ten transactions
[4] Exit the programme
2

Monthly report: 
1992-05: 15.50
1999-12: 89.76
2010-08: 359.89
2015-04: 45.99
2017-04: 23.66
2018-06: 408.99
2018-10: 114.98
2019-05: 612.48
2021-01: 55.89
2022-04: 55.00
2022-11: 4.00
2023-04: 5.33
2024-03: 48.38

Insert a number for the action to be executed:
[1] Enter a transaction
[2] Monthly report
[3] Top ten transactions
[4] Exit the programme
3

The top ten transactio